# Semantic Search on PDF Documents with KDB.AI

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents. 

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

## Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

- How to create vector embeddings using Sentence Transformer
- How to store those embeddings in KDB.AI
- How to search with a query using KDB.AI

## 1. Load and Split Document


In [2]:
%pip install PyPDF2 spacy sentence-transformers kdbai_client -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
#!python3 -m spacy download en_core_web_sm -q
!python3 -m spacy download en_core_web_sm -q

'python3' is not recognized as an internal or external command,
operable program or batch file.


### Load and Split PDF into Sentences

We leverage the power of PyPDF2 for PDF processing and spaCy for advanced natural language processing. The code below extracts content from each page of the PDF and processes it to identify sentences.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way. It is also available on our [GitHub](https://github.com/KxSystems/kdbai-notebooks/tree/main/notebooks/samples/document-search).

In [4]:
import PyPDF2
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


def split_pdf_into_sentences(pdf_path):
    # Open the PDF file
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Extract text from each page and concatenate
        full_text = ""
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            full_text += page.extract_text()

        # Process the text using spaCy for sentence tokenization
        doc = nlp(full_text)
        sentences = [sent.text for sent in doc.sents]

        return sentences


# Define PDF path
pdf_path = "research_paper.pdf"

# Split the PDF into sentences
pdf_sentences = split_pdf_into_sentences(pdf_path)
len(pdf_sentences)

393

In [6]:
type(pdf_sentences[0])

str

## 2. Create Vector Embeddings 

Next, we use the Sentence Transformers library to create embeddings for our collection of sentences.


### Selecting a Sentence Transformer Model

There are 100+ of different types of Sentence Transformers models available - see [HuggingFace](https://huggingface.co/sentence-transformers) for the full list. The diversity among these primarily stems from variations in their training data. Selecting the ideal model for your needs involves matching the domain and task closely, while also considering the benefits of incorporating larger datasets to enhance scale. 

This tutorial will use the `all-MiniLM-L6-v2` pre-trained model. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks including semantic search which makes it a good choice for our needs.

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

C:\Users\jpandza\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Generate Embeddings

We prepare embeddings by applying the sentence transformer model to our sentences to encode them. The we do some transformation to get this into DataFrame which is the format accepted by KDB.AI.

In [8]:
import numpy as np
import pandas as pd

# Create embeddings
embeddings_array = model.encode(np.array(pdf_sentences))
embeddings_list = embeddings_array.tolist()

print(embeddings_list)

embeddings_df = pd.DataFrame({"vectors": embeddings_list, "sentences": pdf_sentences})
#embeddings_df

[[-0.0655134990811348, -0.09104187786579132, 0.06584443151950836, 0.07120892405509949, 5.672908082487993e-05, 0.006440045777708292, -0.030160238966345787, 0.06314815580844879, 0.012700626626610756, 0.04422919079661369, -0.003107610857114196, -0.08268626034259796, -0.02485417015850544, -0.04061656445264816, -0.03626184165477753, 0.0004850968543905765, 0.002818119479343295, -0.08605621755123138, 0.02949381433427334, 0.03651474043726921, 0.06065846234560013, 0.05660031735897064, 0.012513615190982819, 0.03580660745501518, 0.08700316399335861, -0.08000293374061584, -0.02265809290111065, 0.03170919418334961, 0.034226421266794205, -0.07376086711883545, 0.05425088852643967, 0.11951809376478195, 0.05473992973566055, -0.05223912373185158, 0.03899272158741951, -0.06337931007146835, 0.004162135999649763, -0.021685436367988586, 0.020319070667028427, 0.004533982835710049, 0.032556064426898956, -0.04310683533549309, 0.05256164073944092, 0.06542816758155823, -0.05433289334177971, -0.068736732006073, -

It is important to note the dimension of our embeddings is 384. This will need to match the dimensions we set in the KDB.AI index in the next step. We can easily check this using `len` to count elements in our vector.

In [9]:
embeddings_df

,vectors,sentences
0,"[-0.0655134990811348, -0.09104187786579132, 0....","Draft version August 14, 2023\nTypeset using L..."
1,"[-0.07847042381763458, -0.12151646614074707, 0...","2J. Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Fo..."
2,"[-0.07799109816551208, -0.08398669958114624, 0...",We define a novel framework: firstly to predic...
3,"[-0.05338575690984726, -0.03714738041162491, 0...",We predict the spatial and compositional distr...
4,"[-0.06326983124017715, -0.01720457524061203, 0...",Selecting ISO water mass\nfraction as an examp...
...,...,...
388,"[-0.05012289807200432, -0.1008116751909256, -0...",SDSS-IV is managed by the Astrophysical Resear...
389,"[-0.07460092008113861, -0.06173834204673767, 0...","University of Tokyo, the Korean Participation ..."
390,"[-0.07249358296394348, -0.05101022869348526, -...",Software: NumPy (Harris et al. 2020);
391,"[-0.14471641182899475, -0.11599936336278915, 0...",SciPy (Virtanen et al. 2020); Astropy (Astropy...


## 3. Store Embeddings in KDB.AI

With the embeddings created, we need to store them in a vector database to enable efficient searching. KDB.AI is perfect for this task.

### Connect to KDB.AI Session

To use KDB.AI, you will need two session details - a URL endpoint and an API key. To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI session using `kdbai.Session`. Enter the session URL endpoint and API key details from your KDB.AI Cloud portal below.

In [10]:
import kdbai_client as kdbai
from getpass import getpass

'''
KDBAI_ENDPOINT = input('KDB.AI endpoint: ')
KDBAI_API_KEY = getpass('KDB.AI API key: ')
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)
'''
session = kdbai.Session(api_key="73c70f55e3-zwTcApmr1LHpOkv4sXQMdjTKFl/t7lXARtfpvOtitkzIQN5/YB+sT/EAe99sjuHZ5Y/10O9gSvDTRFh1", endpoint="https://cloud.kdb.ai/instance/wge9zktpsm")

### Define Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have one column called `vectors`.

At this point you will select the index and metric you want to use for searching.

With KDB.AI we have the choice between HNSW (Hierarchical Navigable Small World) and Flat indexing methods. Generally, for semantic search of documents, the HNSW indexing method might be more suitable. Here's why:

- **Search Speed and Approximation**: HNSW is designed for fast approximate nearest neighbour searches. It can efficiently handle high-dimensional data, which is common in natural language processing tasks involving text documents.
- **Semantic Representation**: The Sentence Transformers library, used in this example, generates embeddings that capture semantic meaning. HNSW is well-suited for indexing such embeddings and performing semantic searches.
- **Scalability**: HNSW is scalable and can handle large datasets effectively, making it suitable for applications with a vast number of documents.

HNSW provides approximate search results, meaning that the nearest neighbors might not be exact matches but are close in terms of similarity.

In [11]:
pdf_schema = {
    "columns": [
        {"name": "sentences", "pytype": "str"},
        {
            "name": "vectors",
            "vectorIndex": {"dims": 384, "metric": "L2", "type": "hnsw"},
        },
    ]
}

### Create and Save Table

Use `create_table` to create a table.

In [12]:
#table = session.create_table("pdf", pdf_schema)
table = session.table("pdf")

We can use `query` to see our table exists but is empty.

In [13]:
table.query()

### Add Embeddings to Index

In [13]:
table.insert(embeddings_df)

KDBAIException: Failed to insert data in table named: pdf, because of: Timed out waiting for inserted data to arrive.

Re-running `query` we can now see data has been added.

In [ ]:
table.query()

KDBAIException: Failed to process the query {'table': 'powerpoint'} on table named: powerpoint, because of: Table not found, table=powerpoint.

## 4. Searching with a Query using KDB.AI

Now that the embeddings are stored in KDB.AI, we can perform semantic search using `search`. 

First, we embed our search term using the Sentence Transformer model as before. Then we search our index to return the three most similar vectors.

In [ ]:
search_term = "number of interstellar objects in the milky way"
search_term_vector = model.encode(search_term)
search_term_list = [search_term_vector.tolist()]

results = table.search(search_term_list, n=3)
results

[                                           sentences  \
 0  2J. Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Fo...   
 1  In this work, we develop\nthis method and appl...   
 2  Keywords: Interstellar objects (52), Small Sol...   
 
                                              vectors  __nn_distance  
 0  [-0.07847049832344055, -0.12151646614074707, 0...       0.640911  
 1  [-0.08688259869813919, -0.023635469377040863, ...       0.668817  
 2  [-0.08041016012430191, -0.035552043467760086, ...       0.678942  ]

The results returned from `table.search` show the closest matches along with value of nearest neighbour distances `nn_distance`. Let's print the output so we can see the full sentences.

In [ ]:
pd.set_option("display.max_colwidth", None)
results[0]["sentences"]

0    2J. Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just Group plc, Enterprise House, Bancroft road, Reigate, Surrey RH2 7RP, UK\n4Canadian Institute for Theoretical Astrophysics, University of Toronto, 60 St. George Street, Toronto, ON, M5S 3H8, Canada\n5Dunlap Institute for Astronomy and Astrophysics, University of Toronto, 50 St. George Street, Toronto, ON M5S 3H4, Canada\nABSTRACT\nThe Milky Way is thought to host a huge population of interstellar objects (ISOs), numbering\napproximately 1015pc−3around the Sun, which are formed and shaped by a diverse set of processes\nranging from planet formation to galactic dynamics.
1                                                                                                                       

We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way. Let's try another search term.

In [ ]:
search_term = "how does planet formation occur"
#search_term = "who is the author"
search_term_vector = model.encode(search_term)
search_term_list = [search_term_vector.tolist()]

results = table.search(search_term_list, n=3)
results[0]["sentences"]

0                                                                                                                                                                                                                                                                                                                                             The pop-\nulation’s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the source systems.
1                                                                                                                                                                                                                                                                                                                                                             A protoplanetary disk has to first order the same composition as the star it forms around,\nsince they both form from the same molecular cloud core.
2 

Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way. 